In [1]:
import tensorflow as tf
import sounddevice as sd
import numpy as np
import IPython.display as ipd
import scipy as sp
import scipy.signal
import librosa as li
from preprocessing import Audio

In [2]:
tf.config.set_visible_devices(tf.config.list_physical_devices()[0])
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:XLA_CPU:0', device_type='XLA_CPU')]

In [3]:
sd.default.samplerate = 16000
sd.default.channels = 1

In [4]:
@tf.function
def pad(data):
    zero_padding = tf.zeros((16000 - tf.shape(data)[0], 1), dtype=tf.float32)
    return tf.concat([data, zero_padding], 0)

def pad_old(data):
    return tf.keras.preprocessing.sequence.pad_sequences(
        [data],
        maxlen=int(16000 * 1.0),
        padding='post',
        truncating='post',
        dtype='float32'
    )[0]

In [5]:
def get_energy(x, sr=16000):
    x = tf.keras.preprocessing.sequence.pad_sequences(
        [data],
        maxlen=int(16000 * 1.0),
        padding='post',
        truncating='post',
        dtype='float32'
    )[0]
    x = tf.reshape(x, (sr))
    coeff = sp.signal.firwin(999, [260, 700], fs=sr, pass_zero=False)
    x_filtered = sp.signal.lfilter(coeff, 1.0, x)
    x_normalized = x_filtered / x_filtered.max()
    x_squared = np.square(x_normalized)
    splited = np.array_split(x_squared, 200)
    e_parts = np.empty((0))
    for part in splited:
        e_parts = np.append(e_parts, sp.integrate.simps(part))
    return e_parts

In [6]:
path_to_model = 'models/v4/MiraGRUV4.3.2-1.h5'
model = tf.keras.models.load_model(path_to_model)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.Recall(name='recall'),
        tf.keras.metrics.Precision(name='precision'),
    ]
)
model.summary()

Model: "MiraGRUV4.3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization_2 (Normalizati (None, 200, 1)            3         
_________________________________________________________________
gru_22 (GRU)                 (None, 200)               121800    
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 201       
Total params: 122,004
Trainable params: 122,001
Non-trainable params: 3
_________________________________________________________________


In [7]:
data = sd.rec(int(1.0 * sd.default.samplerate), blocking=True)

In [78]:
class_sum = 0
parts = 0
prediction_sum = 0

def predictions(model, data):
    predictions = []
    for parts in range(0, 16000, 4000):
        e_parts = get_energy(data[parts:])
        predictions.append(model(np.reshape(e_parts, (1, 200)))[0][0].numpy())
    return predictions

def callback(indata, outdata, frames, time, status):
    # outdata[:] = indata
    global data, model, class_sum, parts, prediction_sum
    
    parts += 1
    data = data[frames:]
    data = np.append(data, indata, axis=0)
    preds = predictions(model, data)
    
    ipd.clear_output(wait=True)
    
    prediction = tf.math.reduce_mean(preds)
    print(prediction)
    if prediction > 0.5:
        sd.play(data)
        print('Mira {:.3f}'.format(prediction))
            
    
block = int(sd.default.samplerate * 1.0)
stream = sd.Stream(blocksize=block, callback=callback)

In [79]:
stream.start()

tf.Tensor(2.650164e-06, shape=(), dtype=float32)


In [80]:
stream.stop()

tf.Tensor(0.013859242, shape=(), dtype=float32)


In [60]:
data.shape

(44100, 1)

In [32]:
model.input_shape

(None, 200, 1)

In [8]:
tf.math.reduce_mean([1., 2.]).numpy()

1.5

In [43]:
parts = int(16000 * 0.15 * 2)
sd.play(data[parts:])

In [44]:
%%time
e_parts = get_energy(np.reshape(data[parts:], (16000-parts)))
model(np.reshape(e_parts, (1, 200)))

Wall time: 113 ms


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00016987]], dtype=float32)>

In [419]:
predictions(model, data)

[0.0022153258, 0.0071291924, 0.011405498, 0.057645947]

In [45]:
%%time
predictions(model, data)

Wall time: 211 ms


[0.00016987324, 0.00016987324]

In [34]:
%%time
pad(data[parts:])

Wall time: 15.7 ms


<tf.Tensor: shape=(16000, 1), dtype=float32, numpy=
array([[ 0.0000000e+00],
       [ 0.0000000e+00],
       [-3.0517578e-05],
       ...,
       [-6.0729980e-03],
       [-9.3383789e-03],
       [-1.1871338e-02]], dtype=float32)>

In [36]:
%%time
pad_old(data[parts:])


Wall time: 0 ns


array([0.], dtype=float32)